In [1]:
import yaml

path = "./configs/config.yaml"

with open(path, "r") as f:
    config = yaml.safe_load(f)

In [2]:
from src.data_utils import process_data

cleaned = process_data(config["data"]["data_dir"], config["data"]["raw_path"], config["data"]["processed_path"])

Загрузка/обработка данных...
--Данные загружены из кэша


In [3]:
from src.tokenizer import train_tokenizer

train_tokenizer(config["data"]["processed_path"], config["data"]["tokenizer_file_name"])

Токенайзер уже прошел тренировку


In [ ]:
from src.data_utils import data_tokenization

tokenized = data_tokenization(config["data"]["data_dir"], config["data"]["tokenized_file_name"], cleaned)

In [6]:
from src.data_utils import data_split

train_seqs, val_seqs, test_seqs = data_split(tokenized)

📊 Train: 1277520, Val: 159690, Test: 159691


In [7]:
from src.data_utils import TweeterDataset

train_dataset = TweeterDataset(train_seqs)
val_dataset = TweeterDataset(val_seqs)
test_dataset = TweeterDataset(test_seqs)

In [ ]:
from torch.utils.data import DataLoader
from src.data_utils import collate_fn

train_dataloader = DataLoader(
    train_dataset,
    batch_size=config["data"]["train_batch_size"],
    shuffle=True,
    collate_fn=collate_fn,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=config["data"]["val_batch_size"],
    collate_fn=collate_fn,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=config["data"]["test_batch_size"],
    collate_fn=collate_fn,
)

In [ ]:
from src.lstm_train import training, validation, evaluate_rouge_on_val, generate
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file(config["data"]["tokenizer_file_name"])

num_epochs = config["training"]["epochs"]
for epoch in range(num_epochs):
    print(f"Эпоха {epoch+1}/{num_epochs}")
    
    training(train_dataloader)
    should_stop = validation(val_dataloader)
    if should_stop: break

    if (epoch + 1) % 5 == 0 or epoch == 0:
        evaluate_rouge_on_val(test_dataloader, tokenizer)
    
    if (epoch + 1) % 2 == 0 or epoch == 0:
        generate(tokenizer)

In [10]:
from src.eval_lstm import lsmt_generate
prompt = "still dont know"
lsmt_generate(prompt)

'<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7f570d42f1f0>>' → still dont know what to do


In [13]:
from transformers import pipeline
from src.eval_transformer_pipeline import transformer_generate
from src.eval_transformer_pipeline import rouge_calc_for_transformer

# Примеры генерации трансформера
generation_prompts = ["I am about", "What is going on with", "Don't you mind if", "Let us deal with", "Deep Learning is"]
generator = pipeline("text-generation", model="distilgpt2")

transformer_generate(generator, generation_prompts)
rouge_calc_for_transformer(generator)

Loading weights: 100%|██████████| 76/76 [00:00<00:00, 595.35it/s, Materializing param=transformer.wte.weight]            


Примеры генерации:
1) I am about to be released by the company.”  The first episode of the series is called �
2) What is going on with this, and what do you think this is going to be?
3) Don't you mind if I'm on the phone with you?     If you're ready, ask my
4) Let us deal with the question: Who exactly does the money come from?    So, even if we
5) Deep Learning is a tool for learning new concepts. These resources are available as a single sheet of data. Each sheet


100%|██████████| 486/486 [00:09<00:00, 52.43it/s]


ROUGE-1: 0.0497
